In [14]:
# pip install pandas langchain langchain-core langchain-community tenacity
import json
import re
import pandas as pd
from typing import List, Dict, Any

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama

# -----------------------------
# 1) Build the chain (Ollama)
# -----------------------------
def build_chain(model_name: str = "llama3.2:3b", temperature: float = 0.0):
    """
    Composed chain = Prompt → LLM → String parser, with retries attached later.
    """
    prompt = ChatPromptTemplate.from_messages([
        ("system",
         "Tu es un assistant d'extraction d'information clinique. "
         "Étant donné un compte-rendu et une LISTE DE CONCEPTS contrôlée, "
         "tu dois renvoyer EXCLUSIVEMENT un TABLEAU JSON valide (aucun autre texte) où chaque item a :\n"
         "- concept: string (exactement un des concepts fournis)\n"
         "- context: string (justification courte du rapport, '' si aucune)\n"
         "- presence: string ∈ {{'True','False','pas mentionné'}}\n"
         "Règles :\n"
         "• Si le rapport nie un concept ⇒ presence='False'.\n"
         "• S'il n'est pas référencé ⇒ 'pas mentionné'.\n"
         "• N'inclus AUCUN concept en dehors de la liste fournie.\n"
         "• La sortie DOIT être un JSON VALIDE (un tableau)."),
        ("user",
         "Concepts: {concept_list}\n\nRapport:\n{report_text}\n\n"
         "Rappels: renvoie UNIQUEMENT un tableau JSON (ex: [{{...}},{{...}}]).")
    ])

    llm = ChatOllama(model=model_name, temperature=temperature, base_url="http://lx181.intra.chu-rennes.fr:11434/")
    chain = prompt | llm | StrOutputParser()
    # Add retry (2 attempts total) for transient format issues
    chain = chain.with_retry(stop_after_attempt=2, wait_exponential_jitter=True)
    return chain

# -----------------------------
# 2) Utilities
# -----------------------------
def concepts_from_columns(df: pd.DataFrame) -> List[str]:
    base = ["Id", "Report"]
    return [c for c in df.columns if c not in base]

def normalize_presence(value: Any) -> bool:
    """
    Map 'presence' to boolean:
      True/'True'/true → True
      'False'/False/'pas mentionné' → False
    """
    if isinstance(value, bool):
        return value
    if isinstance(value, str):
        v = value.strip().lower()
        if v in {"true", "vrai", "présent", "present"}:
            return True
        if v in {"false", "faux", "absent"}:
            return False
        if "pas mentionné" in v or "pas mentionne" in v:
            return None
    return False

def safe_parse_json(raw: str) -> List[Dict[str, Any]]:
    """
    Robust JSON parse:
    - First try json.loads as-is
    - If it fails, extract the first [...] block and try again
    - Return [] if everything fails
    """
    if not isinstance(raw, str):
        return []
    try:
        data = json.loads(raw)
        return data if isinstance(data, list) else []
    except Exception:
        pass

    # Try to extract the first JSON array
    m = re.search(r"\[.*\]", raw, flags=re.DOTALL)
    if m:
        candidate = m.group(0)
        try:
            data = json.loads(candidate)
            return data if isinstance(data, list) else []
        except Exception:
            return []
    return []

# -----------------------------
# 3) Core runner
# -----------------------------
def extract_and_project_best(df: pd.DataFrame,
                             model_name: str = "llama3.1",
                             temperature: float = 0.0,
                             max_concurrency: int = 4) -> pd.DataFrame:
    """
    - Runs Ollama via LangChain to extract [{concept, context, presence}, ...] per report.
    - Projects to the same schema as input: [Id, Report, <concept_1> ...] with booleans.
    - Uses chain.batch() for bounded concurrency.
    """
    if "Id" not in df.columns or "Report" not in df.columns:
        raise ValueError("Input DataFrame must contain 'Id' and 'Report' columns.")

    concepts = concepts_from_columns(df)
    chain = build_chain(model_name=model_name, temperature=temperature)

    # Prepare inputs for batch
    inputs = [{
        "concept_list": ", ".join(concepts),
        "report_text": str(row["Report"])
    } for _, row in df.iterrows()]

    # Batched calls with bounded concurrency
    raw_outputs: List[str] = chain.batch(inputs, config={"max_concurrency": max_concurrency})

    # Initialize final DataFrame (copy Id & Report, default all concepts to False)
    final = df[["Id", "Report"]].copy()
    for c in concepts:
        final[c] = False

    # Project each row's extraction to booleans
    for idx, raw in enumerate(raw_outputs):
        items = safe_parse_json(raw)

        # Consolidate by concept: True if any item says True
        by_concept: Dict[str, bool] = {}
        for it in items if isinstance(items, list) else []:
            concept = it.get("concept", "")
            presence = normalize_presence(it.get("presence", "pas mentionné"))
            if concept in concepts:
                by_concept[concept] = by_concept.get(concept, False) or presence

        for c in concepts:
            final.iat[idx, final.columns.get_loc(c)] = bool(by_concept.get(c, False))

    return final

In [15]:
# -----------------------------
# 4) Example usage
# -----------------------------
df = pd.DataFrame([
    {
        "Id": 1,
        "Report": "Patiente avec ostéoporose traitée par bisphosphonates depuis 2019. "
                  "Fracture du poignet droit confirmée à la radiographie. Pas d'asthme connu."
    },
    {
        "Id": 2,
        "Report": "Angor atypique suspecté. Aucun antécédent de fracture ou d'asthme."
    },
])
# Ensure your concept columns exist (can be empty at first)
for c in ["Asthme", "Angor", "Ostéoporose", "Fracture"]:
    if c not in df.columns:
        df[c] = None

result = extract_and_project_best(df, model_name="llama3.2:3b", temperature=0.0, max_concurrency=4)
print(result)


   Id                                             Report  Asthme  Angor  \
0   1  Patiente avec ostéoporose traitée par bisphosp...   False  False   
1   2  Angor atypique suspecté. Aucun antécédent de f...   False   True   

   Ostéoporose  Fracture  
0         True      True  
1        False     False  


In [18]:
print("Tu es un assistant d'extraction d'information clinique. "
         "Étant donné un compte-rendu et une LISTE DE CONCEPTS contrôlée, "
         "tu dois renvoyer EXCLUSIVEMENT un TABLEAU JSON valide (aucun autre texte) où chaque item a :\n"
         "- concept: string (exactement un des concepts fournis)\n"
         "- context: string (justification courte du rapport, '' si aucune)\n"
         "- presence: string ∈ {{'True','False','pas mentionné'}}\n"
         "Règles :\n"
         "• Si le rapport nie un concept ⇒ presence='False'.\n"
         "• S'il n'est pas référencé ⇒ 'pas mentionné'.\n"
         "• N'inclus AUCUN concept en dehors de la liste fournie.\n"
         "• La sortie DOIT être un JSON VALIDE (un tableau).")

Tu es un assistant d'extraction d'information clinique. Étant donné un compte-rendu et une LISTE DE CONCEPTS contrôlée, tu dois renvoyer EXCLUSIVEMENT un TABLEAU JSON valide (aucun autre texte) où chaque item a :
- concept: string (exactement un des concepts fournis)
- context: string (justification courte du rapport, '' si aucune)
- presence: string ∈ {{'True','False','pas mentionné'}}
Règles :
• Si le rapport nie un concept ⇒ presence='False'.
• S'il n'est pas référencé ⇒ 'pas mentionné'.
• N'inclus AUCUN concept en dehors de la liste fournie.
• La sortie DOIT être un JSON VALIDE (un tableau).
